In [1]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv"
)
df.head()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
# df.drop(columns=["id", "Unnamed: 32"], inplace=True)
df.head()


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2
)


In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [13]:
X_train


array([[-0.60996968,  0.5468645 , -0.62198382, ..., -0.63838376,
        -0.74055794,  0.07387081],
       [-0.58123378, -0.30491372, -0.64028295, ..., -1.32590811,
        -1.16589612, -0.74444195],
       [-1.15595178,  0.48771323, -1.17678005, ..., -1.35942705,
        -0.88233733, -0.18729284],
       ...,
       [ 0.51647759,  1.11944875,  0.50798721, ..., -0.0293277 ,
        -0.14954496, -1.15251015],
       [ 1.10843713,  0.42619592,  1.36887792, ...,  1.27590276,
         0.54501476,  1.94499952],
       [-0.51226763,  2.54381122, -0.58496968, ..., -1.75779265,
        -0.78675572, -0.90440469]], shape=(455, 30))

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


In [ ]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train).to(torch.float64)
y_test_tensor = torch.from_numpy(y_test).to(torch.float64)


In [83]:
class MyNNModule(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1, dtype=torch.float64)
        self.sigmoid = nn.Sigmoid()
        self.loss_function = nn.BCELoss()

    def forward(self, features):
        z = self.linear(features)
        y_pred = self.sigmoid(z)
        return y_pred

    # def loss_function(self, y_pred, y_actual):
    #     epsilon = 1e-7
    #     y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    #     loss = -(
    #         y_actual * torch.log(y_pred) + (1 - y_actual) * torch.log(1 - y_pred)
    #     ).mean()
    #     return loss


In [86]:
learning_rate = 0.1
epochs = 25


In [84]:
model = MyNNModule(X_train_tensor.shape[1])


In [90]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [91]:
for epoch in range(epochs):
    y_pred = model(X_train_tensor)

    loss = model.loss_function(y_pred, y_train_tensor.view(-1, 1))

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss}")


Epoch: 1, Loss: 0.12036976470118045
Epoch: 2, Loss: 0.11974243676074975
Epoch: 3, Loss: 0.11913044208555827
Epoch: 4, Loss: 0.11853318180464789
Epoch: 5, Loss: 0.11795008887854842
Epoch: 6, Loss: 0.11738062597014992
Epoch: 7, Loss: 0.11682428348662527
Epoch: 8, Loss: 0.11628057777639211
Epoch: 9, Loss: 0.11574904946681211
Epoch: 10, Loss: 0.11522926192983059
Epoch: 11, Loss: 0.11472079986408597
Epoch: 12, Loss: 0.11422326798319153
Epoch: 13, Loss: 0.1137362898009292
Epoch: 14, Loss: 0.11325950650501643
Epoch: 15, Loss: 0.11279257591192521
Epoch: 16, Loss: 0.11233517149595905
Epoch: 17, Loss: 0.11188698148644453
Epoch: 18, Loss: 0.11144770802747181
Epoch: 19, Loss: 0.11101706639513698
Epoch: 20, Loss: 0.11059478426770426
Epoch: 21, Loss: 0.11018060104452039
Epoch: 22, Loss: 0.10977426720988874
Epoch: 23, Loss: 0.1093755437384465
Epoch: 24, Loss: 0.10898420153889098
Epoch: 25, Loss: 0.10860002093317385
